In [1]:
!pip install chromadb

In [2]:
!pip install unstructured selenium

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 34.1.0 requires urllib3<2.4.0,>=1.24.2, but you have urllib3 2.5.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
google-adk 1.16.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which

In [3]:
!pip install -U langchain-community

In [4]:
!pip install -U langchain-openai

In [5]:
#!pip install deeplake[enterprise]==3.9.3

In [6]:
#!pip install deeplake[enterprise]

In [5]:
!pip install langchain-chroma

  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.38.0 requires urllib3[socks]<3.0,>=2.5.0, but you have urllib3 2.3.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
google-adk 1.16.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 whi

In [10]:
!pip install -U langchain-openai

In [11]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
#from openai import OpenAI
#from langchain.vectorstores import DeepLake
#import deeplake
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
#from langchain import OpenAI
from langchain_openai import OpenAI
from langchain_chroma import Chroma
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate
from google.colab import userdata
from dotenv import load_dotenv
import os
import openai
load_dotenv()

False

In [12]:
openai_api_key = userdata.get('OPENAI_API_KEY').strip()
os.environ['OPENAI_API_KEY'] = openai_api_key

In [49]:
from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [23]:
#org_id = userdata.get('ORG_ID').strip()

In [14]:
# List of articles that we will use

urls = [
    'https://beebom.com/what-is-nft-explained/',
    'https://beebom.com/how-delete-spotify-account/',
    'https://beebom.com/how-download-gif-twitter/',
    'https://beebom.com/how-use-chatgpt-linux-terminal/',
    'https://beebom.com/how-delete-spotify-account/',
    'https://beebom.com/how-save-instagram-story-with-music/',
    'https://beebom.com/how-install-pip-windows/',
    'https://beebom.com/how-check-disk-usage-linux/'
]

In [15]:
# Loading the document
loader =SeleniumURLLoader(urls=urls)
docs_not_splitted =loader.load()
text_splitter =CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs =text_splitter.split_documents(docs_not_splitted)

In [16]:
embeddings =OpenAIEmbeddings(model="text-embedding-ada-002")

In [17]:
#We are going to check if the database exists, in case it exists we delete the database
db_name = "vector_db"
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
    print(db_name)



vector_db


In [18]:
#We are going to create our database and add our documents
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 107 documents


In [19]:
query = "how to check disk usage in linux?"
query_embedding = embeddings.embed_query(query)

In [20]:
results = vectorstore.similarity_search_by_vector(query_embedding, k=2)

In [21]:
# Here is one of the results who match the most with our query
results[0]

Document(id='5de35e81-5c8c-4448-86ab-41090eb624bb', metadata={'description': 'In this guide, we show you the best ways to check disk usage on Linux using GUI tools as well as CLI commands like du and df.', 'title': 'How to Check Disk Usage in Linux (4 Methods) | Beebom', 'language': 'en-US', 'source': 'https://beebom.com/how-check-disk-usage-linux/'}, page_content='Home > Tech > How to Check Disk Usage in Linux (4 Methods)\n\nHow to Check Disk Usage in Linux (4 Methods)\n\nBeebom Staff\n\nUpdated: October 15, 2025\n\nComments 0\n\nCopied\n\nThere may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.\n\nCh

In [22]:
# This our prompt template for customer chatbot
# Answer question by only using information from our database

template ="""You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""
prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,)



In [23]:
# Here we are going to retrieve all relevants chunks from  our dataset
retrieved_chunks = [doc.page_content for doc in results]

In [24]:
# Format the prompt to meet the requirement of our template

chunks_formatted ="\n\n".join(retrieved_chunks)
prompt_formatted =prompt.format(chunks_formatted=chunks_formatted, query=query)

In [51]:
response = llm_openai.invoke(prompt_formatted)

In [52]:
response

AIMessage(content='To check disk usage in Linux, you can use the df command in the terminal or the Disk Usage Analyzer tool in the GUI. The df command is a common way to check current disk usage and available disk space in Linux. You can also use the Disk Usage Analyzer tool to visually see disk occupancy for all folders and easily identify which ones are taking up the most space.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 464, 'total_tokens': 538, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CUyLXknDq3EHr6eB2V2CKmJ0ouQpk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a0647eb2-57c2-452d-be1e-abde9bee7618-0', usag